In [28]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# read the files changes path to where the data is stored
test = pd.read_csv(r"C:\Users\youpz\Documents\Master\P5\Data mining techniques\Assignment2\data\dmt-2025-2nd-assignment\test_set_VU_DM.csv")
train = pd.read_csv(r"C:\Users\youpz\Documents\Master\P5\Data mining techniques\Assignment2\data\dmt-2025-2nd-assignment\training_set_VU_DM.csv")
sample = pd.read_csv(r"C:\Users\youpz\Documents\Master\P5\Data mining techniques\Assignment2\data\dmt-2025-2nd-assignment\submission_sample.csv")

In [3]:
#number op srch_id
print(len(sample['srch_id'].unique()), 'number of search IDs in sample')
print(len(sample['prop_id'].unique()), 'number of different hotels in sample')
print(len(test['srch_id'].unique()),'number of search ids in testdataset')

199549 number of search IDs in sample
129438 number of different hotels in sample
199549 number of search ids in testdataset


In [4]:
# Check if srch_ids match between sample, test, and train --> to get a feeling for the code

sample_ids = set(sample['srch_id'].unique())
test_ids = set(test['srch_id'].unique())
train_ids = set(train['srch_id'].unique())

# Check sample vs. test
if sample_ids != test_ids:
    missing_in_sample = test_ids - sample_ids
    missing_in_test = sample_ids - test_ids
    print(f"Mismatch in srch_ids between sample and test! Missing in sample: {len(missing_in_sample)}, Missing in test: {len(missing_in_test)}")
else:
    print("srch_id sets match between sample and test.")

# Check sample vs. train
if sample_ids != train_ids:
    missing_in_sample_train = train_ids - sample_ids
    missing_in_train_sample = sample_ids - train_ids
    print(f"Mismatch in srch_ids between sample and train! Missing in sample: {len(missing_in_sample_train)}, Missing in train: {len(missing_in_train_sample)}")
else:
    print("srch_id sets match between sample and train.")



srch_id sets match between sample and test.
Mismatch in srch_ids between sample and train! Missing in sample: 79881, Missing in train: 79635


In [5]:
def missing_values_table(df):
    """Returns a DataFrame with missing counts and percent missing for each column."""
    n = len(df)
    missing_count   = df.isna().sum()
    missing_percent = 100 * missing_count / n
    missing_df = (
        pd.DataFrame({
            'missing_count':   missing_count,
            'missing_percent': missing_percent
        })
        .sort_values('missing_percent', ascending=False)
    )
    return missing_df

In [6]:
mv = missing_values_table(train)
print(mv.head(20))       # top 20 most‐missing columns
# Or to filter down to “lots” of missing, say >30%:
print(mv[mv['missing_percent'] > 30])

                           missing_count  missing_percent
comp1_rate_percent_diff          4863908        98.095353
comp6_rate_percent_diff          4862173        98.060362
comp1_rate                       4838417        97.581250
comp1_inv                        4828788        97.387053
comp4_rate_percent_diff          4827261        97.356256
gross_bookings_usd               4819957        97.208949
comp7_rate_percent_diff          4819832        97.206428
comp6_rate                       4718190        95.156511
visitor_hist_starrating          4706481        94.920364
visitor_hist_adr_usd             4705359        94.897735
comp6_inv                        4697371        94.736633
comp4_rate                       4650969        93.800797
comp7_rate                       4642999        93.640058
srch_query_affinity_score        4640941        93.598552
comp4_inv                        4614684        93.069001
comp7_inv                        4601925        92.811677
comp3_rate_per

In [7]:
########################## some beginning on the feature engineering


def preprocess_missing_and_competitors(train_df, test_df):
    # 1) Drop features with >93% missing or that leak the target
    drop_cols = [
        # competitor 1,4,6,7 are ~97–98% missing → too sparse to learn
        *[f'comp{i}_{t}' for i in [1,4,6,7] for t in ['rate','inv','rate_percent_diff']],
        'gross_bookings_usd'  # only in train, leaks booking price
    ]
    train_df.drop(columns=drop_cols, errors='ignore', inplace=True)
    test_df.drop(columns=drop_cols, errors='ignore', inplace=True)

    # 2) Impute & flag user history features
    #    Missing means “no prior purchases” → keep with sentinel + flag
    for df in (train_df, test_df):
        # visitor_hist_starrating
        df['hist_star_na'] = df['visitor_hist_starrating'].isna().astype(int)
        # fill with median starrating across users
        star_med = train_df['visitor_hist_starrating'].median()
        df['visitor_hist_starrating'] = df['visitor_hist_starrating'].fillna(star_med)

        # visitor_hist_adr_usd (avg USD spend)
        df['hist_adr_na'] = df['visitor_hist_adr_usd'].isna().astype(int)
        adr_med = train_df['visitor_hist_adr_usd'].median()
        df['visitor_hist_adr_usd'] = df['visitor_hist_adr_usd'].fillna(adr_med)

    # 3) Impute & flag affinity score
    #    Null means “hotel never seen” → fill with global minimum and flag
    affinity_min = train_df['srch_query_affinity_score'].min(skipna=True)
    for df in (train_df, test_df):
        df['affinity_na'] = df['srch_query_affinity_score'].isna().astype(int)
        df['srch_query_affinity_score'] = (
            df['srch_query_affinity_score']
            .fillna(affinity_min)
        )

    # 4) Keep & impute competitor 2,3,5,8 features (~50–90% missing)
    #    Null → “no data” sentinel (for categorical) or 0 (for percent diff), plus flag
    keep_comps = [2,3,5,8]
    for i in keep_comps:
        # availability flag
        inv_col = f'comp{i}_inv'
        flag_col = f'comp{i}_inv_na'
        for df in (train_df, test_df):
            df[flag_col] = df[inv_col].isna().astype(int)
            # fill null with 2 (new category: 0=no avail,1=avail,2=no data)
            df[inv_col] = df[inv_col].fillna(2).astype(int)

        # price‐compare flag
        rate_col = f'comp{i}_rate'
        rate_flag = f'comp{i}_rate_na'
        for df in (train_df, test_df):
            df[rate_flag] = df[rate_col].isna().astype(int)
            # fill null as “no data” = 2
            df[rate_col] = df[rate_col].fillna(2).astype(int)

        # percent_diff
        pdiff_col = f'comp{i}_rate_percent_diff'
        pdiff_flag = f'comp{i}_pdiff_na'
        for df in (train_df, test_df):
            df[pdiff_flag] = df[pdiff_col].isna().astype(int)
            # fill null as 0% diff (no info)
            df[pdiff_col] = df[pdiff_col].fillna(0.0)

    # 5) Bucket orig_destination_distance
    #    Missing → sentinel bucket + flag
    for df in (train_df, test_df):
        df['dist_na'] = df['orig_destination_distance'].isna().astype(int)
        df['orig_destination_distance'] = (
            df['orig_destination_distance'].fillna(-1)
        )
        # define bins (in km)
        bins = [-1, 0, 10, 50, 200, np.inf]
        labels = ['missing','0-10km','10-50km','50-200km','200km+']
        df['dist_bucket'] = pd.cut(
            df['orig_destination_distance'],
            bins=bins, labels=labels
        )

    return train_df, test_df

def create_base_features(df):
    """1) Parse datetime & basic price/historical features."""
    df = df.copy()
    # --- Date/time splits ---
    df['date_time']   = pd.to_datetime(df['date_time'])
    df['search_year'] = df['date_time'].dt.year
    df['search_month']= df['date_time'].dt.month
    df['search_day']  = df['date_time'].dt.day
    df['search_hour'] = df['date_time'].dt.hour

    # --- Price per night & hist price devation ---
    df['price_per_night'] = df['price_usd'] / df['srch_length_of_stay']
    # fill missing visitor_hist_adr with median later in preprocessing
    return df


def add_destination_stats(train_df, test_df):
    """6) Dest‑level total searches & booking rate."""
    dest = (
        train_df
        .groupby('srch_destination_id')
        .agg(dest_searches=('srch_id','count'),
             dest_bookings=('booking_bool','sum'))
        .assign(dest_booking_rate=lambda x: x.dest_bookings / x.dest_searches)
        .reset_index()
    )
    # Reassign merge result back to each DataFrame
    train_df = train_df.merge(
        dest[['srch_destination_id','dest_searches','dest_booking_rate']],
        on='srch_destination_id', how='left'
    )
    test_df = test_df.merge(
        dest[['srch_destination_id','dest_searches','dest_booking_rate']],
        on='srch_destination_id', how='left'
    )
    return train_df, test_df

def add_within_search_features(df):
    """7) Z‑scores & deltas in each search group."""
    grp = df.groupby('srch_id')
    # price
    df['price_mean_srch'] = grp['price_usd'].transform('mean')
    df['price_std_srch']  = grp['price_usd'].transform('std').fillna(1)
    df['price_zscore']    = (df['price_usd'] - df['price_mean_srch']) / df['price_std_srch']
    # stars
    df['star_mean_srch']  = grp['prop_starrating'].transform('mean')
    df['star_delta_srch'] = df['prop_starrating'] - df['star_mean_srch']
    # user delta
    df['star_delta_user'] = df['prop_starrating'] - df['visitor_hist_starrating']
    # distance
    df['dist_mean_srch']  = grp['orig_destination_distance'].transform('mean')
    df['dist_std_srch']   = grp['orig_destination_distance'].transform('std').fillna(1)
    df['dist_zscore']     = (df['orig_destination_distance'] - df['dist_mean_srch']) / df['dist_std_srch']
    return df

def add_temporal_features(df):
    """8) Weekday/weekend & check‑in weekend flags."""
    # day‑of‑week for search
    df['search_dow'] = df['date_time'].dt.weekday  # 0=Mon…6=Sun
    df['is_search_weekend'] = df['search_dow'].isin([5,6]).astype(int)
    # approximate check‑in day
    checkin = df['date_time'] + pd.to_timedelta(df['srch_booking_window'], 'D')
    df['checkin_dow'] = checkin.dt.weekday
    df['is_checkin_weekend'] = df['checkin_dow'].isin([5,6]).astype(int)
    return df

def add_ranks(df):
    """9) Dense ranks of price, star & distance within each search."""
    df['price_rank'] = df.groupby('srch_id')['price_usd'].rank('dense', ascending=True)
    df['star_rank']  = df.groupby('srch_id')['prop_starrating'].rank('dense', ascending=False)
    df['dist_rank']  = df.groupby('srch_id')['orig_destination_distance'].rank('dense', ascending=True)
    return df

In [8]:
train_feat, test_feat = preprocess_missing_and_competitors(train, test)

# create_base_features returns a new DataFrame, so we need to capture it
train_feat = create_base_features(train_feat)
test_feat  = create_base_features(test_feat)

train_feat, test_feat = add_destination_stats(train_feat, test_feat)

train_feat = add_within_search_features(train_feat)
train_feat = add_temporal_features(train_feat)
train_feat = add_ranks(train_feat)

test_feat = add_within_search_features(test_feat)
test_feat = add_temporal_features(test_feat)
test_feat = add_ranks(test_feat)

In [9]:
############################################### create train and test data

# Create relevance label for training
y = train_feat['booking_bool'] * 5 + train_feat['click_bool']

# List of features to drop (train-only or unneeded)
drop_cols = ['date_time','click_bool','booking_bool','gross_bookings_usd','position']
features = [c for c in train_feat.columns if c not in drop_cols + ['srch_id','prop_id']]

#create X train and X_test
X = train_feat[features]
X_test = test_feat[features]


# Extract unique query IDs so we can split at the search-session level
group_ids = train_feat['srch_id'].unique()

# train test split 80/20
train_ids, valid_ids = train_test_split(
    group_ids,
    test_size=0.2,
    random_state=22
)

# Build boolean masks that mark every row in train_feat as belonging
#    either to the train split or the validation split, based on its srch_id
mask_tr = train_feat['srch_id'].isin(train_ids)
mask_va = train_feat['srch_id'].isin(valid_ids)

# Subset your feature matrix X and label vector y according to those masks
#    so X_tr/y_tr and X_va/y_va correspond to disjoint sets of queries
X_tr, y_tr = X[mask_tr], y[mask_tr]
X_va, y_va = X[mask_va], y[mask_va]

# Compute the “group sizes” for LightGBM’s ranker: for each query (srch_id),
#    count how many candidate rows belong to it.  This array of counts
#    tells the ranker where one query ends and the next begins.
groups_tr = train_feat[mask_tr].groupby('srch_id').size().values
groups_va = train_feat[mask_va].groupby('srch_id').size().values

#get train and valdiation data to train the model
train_data = lgb.Dataset(X_tr, label=y_tr, group=groups_tr)
valid_data = lgb.Dataset(X_va, label=y_va, group=groups_va)

In [10]:
#run lgb model and set paramters

params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [5],
    'learning_rate': 0.05,
    'num_leaves': 64,
    'verbose': -1
}

#dont make the model run endlesly 
model = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)],
    num_boost_round=1000
)


Training until validation scores don't improve for 50 rounds
[100]	valid_0's ndcg@5: 0.378551
[200]	valid_0's ndcg@5: 0.385603
[300]	valid_0's ndcg@5: 0.388075
[400]	valid_0's ndcg@5: 0.389296
Early stopping, best iteration is:
[437]	valid_0's ndcg@5: 0.389508


In [11]:
# Predict relevance scores for each test row
preds = model.predict(X_test)

# Insert those scores into the sample submission DataFrame
sample['score'] = preds

#  Sort by search session (ascending) and score (descending)
#    so that for each srch_id, the most relevant prop_id comes first
submission = sample.sort_values(
    ['srch_id', 'score'],
    ascending=[True, False]
)

# keep only the required columns and write to CSV
#    Kaggle expects: srch_id, prop_id (in ranked order)
submission[['srch_id', 'prop_id']].to_csv(
    'submission.csv',
    index=False
)
print("Submission.csv adjusted with new scores!")

Submission.csv adjusted with new scores!
